In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy.special import erf

# Parâmetros
L = 1.0
u = 1.5
c0 = 1.5
D_values = [1e-5, 1e-3, 1e-1]  # Diferentes valores de coeficiente de difusão
x_min, x_max = -10, 10
num_points = 1000
times = [0, 1, 2, 3, 4, 5]

# Condição inicial
def initial_condition(x):
    return np.where(np.abs(x) <= L, c0, 0)

# Malha espacial
x_full = np.linspace(x_min, x_max, num_points)

# Solução analítica para advecção
def advection_solution(x, t):
    return initial_condition(x - u * t)

# Solução analítica para advecção-difusão
def advection_diffusion_solution(x, t, D):
    factor = np.sqrt(4 * D * t)
    result = np.zeros_like(x)
    for i, xi in enumerate(x):
        integral_limit_plus = (L + (xi - u * t)) / factor
        integral_limit_minus = (L - (xi - u * t)) / factor
        result[i] = c0 / 2 * (erf(integral_limit_plus) + erf(integral_limit_minus))
    return result

# Gerar gráficos e salvar tabelas
for D in D_values:
    for t in times:
        # Ajustar x selecionado conforme o movimento da advecção
        x_selected = np.linspace(-L + u * t - 5, L + u * t + 5, 9)  # Centrado em torno da advecção

        # Calculando soluções
        advection_values = advection_solution(x_selected, t)
        advection_diffusion_values = advection_diffusion_solution(x_selected, t, D)

        # Plotando e salvando gráficos
        plt.figure(figsize=(8, 5))
        plt.plot(x_full, advection_solution(x_full, t), linestyle='--', label='Advecção')
        plt.plot(x_full, advection_diffusion_solution(x_full, t, D), label='Advecção-Difusão')
        plt.title(f'Solução para D={D}, t={t}')
        plt.xlabel('x')
        plt.ylabel('Concentração c(x, t)')
        plt.legend()
        plt.grid(True)
        plt.savefig(f'./plot/Advec_Difus_t{t}_D{D}.png')
        plt.close()

        # Criando e salvando tabela
        data = {
            'x': np.round(x_selected, 2),  # Arredondando os valores de x para duas casas decimais
            'Advecção': np.round(advection_values, 2),  # Advecção arredondada para duas casas decimais
            'Advecção-Difusão': advection_diffusion_values  # Mantendo a precisão completa para Advecção-Difusão
        }
        df = pd.DataFrame(data)
        df.to_csv(f'./table/Tabela_Advec_Difus_t{t}_D{D}.csv', index=False)


/tmp/ipykernel_54443/50733831.py:31: RuntimeWarning: divide by zero encountered in scalar divide
  integral_limit_plus = (L + (xi - u * t)) / factor
/tmp/ipykernel_54443/50733831.py:32: RuntimeWarning: divide by zero encountered in scalar divide
  integral_limit_minus = (L - (xi - u * t)) / factor


In [9]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy.special import erf

def initial_condition(x, L, c0):
    """
    Define a condição inicial para a concentração do soluto.
    
    Parameters:
        x (np.array): Array de posições espaciais.
        L (float): Largura do pulso inicial de concentração.
        c0 (float): Concentração inicial dentro do pulso.
        
    Returns:
        np.array: Concentração inicial em cada posição x.
    """
    return np.where(np.abs(x) <= L, c0, 0)

def advection_solution(x, t, u, L, c0):
    """
    Calcula a solução analítica para a equação de advecção em um dado tempo.
    
    Parameters:
        x (np.array): Array de posições espaciais.
        t (float): Tempo no qual a solução é calculada.
        u (float): Velocidade de advecção.
        L (float): Largura do pulso inicial de concentração.
        c0 (float): Concentração inicial dentro do pulso.
        
    Returns:
        np.array: Concentração em cada posição x no tempo t.
    """
    return initial_condition(x - u * t, L, c0)

def advection_diffusion_solution(x, t, u, D, L, c0):
    """
    Calcula a solução analítica para a equação de advecção-difusão em um dado tempo.
    
    Parameters:
        x (np.array): Array de posições espaciais.
        t (float): Tempo no qual a solução é calculada.
        u (float): Velocidade de advecção.
        D (float): Coeficiente de difusão.
        L (float): Largura do pulso inicial de concentração.
        c0 (float): Concentração inicial dentro do pulso.
        
    Returns:
        np.array: Concentração em cada posição x no tempo t.
    """
    factor = np.sqrt(4 * D * t)
    result = np.zeros_like(x)
    for i, xi in enumerate(x):
        integral_limit_plus = (L + (xi - u * t)) / factor
        integral_limit_minus = (L - (xi - u * t)) / factor
        result[i] = c0 / 2 * (erf(integral_limit_plus) + erf(integral_limit_minus))
    return result

def generate_plots_and_tables(x_full, times, D_values, u, L, c0):
    """
    Gera gráficos e tabelas para advecção e advecção-difusão.
    
    Parameters:
        x_full (np.array): Array completo de posições espaciais para plotagem.
        x_selected (np.array): Array de posições selecionadas para tabelas.
        times (list): Lista de tempos para análise.
        D_values (list): Lista de valores do coeficiente de difusão.
        u (float): Velocidade de advecção.
        L (float): Largura do pulso inicial de concentração.
        c0 (float): Concentração inicial dentro do pulso.
        output_dir (str): Diretório para salvar os outputs.
    """
    for D in D_values:
        for t in times:
            x_center = u * t
            x_selected = np.linspace(x_center - 2.25, x_center + 2.75, 10)  # 10 pontos ao redor do centro do pulso
            
            advection_values = advection_solution(x_selected, t, u, L, c0)
            advection_diffusion_values = advection_diffusion_solution(x_selected, t, u, D, L, c0)

            plt.figure(figsize=(8, 5))
            plt.plot(x_full, advection_solution(x_full, t, u, L, c0), linestyle='--', label='Advecção')
            plt.plot(x_full, advection_diffusion_solution(x_full, t, u, D, L, c0), label='Advecção-Difusão')
            plt.title(f'Solução para D={D}, t={t}')
            plt.xlabel('x')
            plt.ylabel('Concentração c(x, t)')
            plt.legend()
            plt.grid(True)
            plt.savefig(f'./plot/Advec_Difus_t{t}_D{D}.png')
            plt.close()

            data = {
                'x': np.round(x_selected, 2),
                'Advecção': np.round(advection_values, 2),
                'Advecção-Difusão': advection_diffusion_values
            }
            df = pd.DataFrame(data)
            df.to_csv(f'./table/Tabela_Advec_Difus_t{t}_D{D}.csv', index=False)
            df.to_latex(f'./latex/Tabela_Advec_Difus_t{t}_D{D}.tex', index=False, column_format='ccc', header=['x', 'Advecção', 'Advecção-Difusão'])

# Parâmetros
L = 1.0
u = 1.5
c0 = 1.5
D_values = [1e-5, 1e-3, 1e-1]
x_min, x_max = -10, 10
num_points = 1000
times = [0, 1, 2, 3, 4, 5]
x_full = np.linspace(x_min, x_max, num_points)

# Executar função principal
generate_plots_and_tables(x_full, times, D_values, u, L, c0)

/tmp/ipykernel_54443/250930777.py:54: RuntimeWarning: divide by zero encountered in scalar divide
  integral_limit_plus = (L + (xi - u * t)) / factor
/tmp/ipykernel_54443/250930777.py:55: RuntimeWarning: divide by zero encountered in scalar divide
  integral_limit_minus = (L - (xi - u * t)) / factor
